In [ ]:
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

def similarity(a: str, b: str) -> float:
    emb_1 = encoder.encode(a)
    emb_2 = encoder.encode(b)

    return round(float(encoder.similarity(emb_1, emb_2).item()), 3)

dataset = load_dataset('IlyaGusev/librusec_full', split='train', streaming=True)

with open('All_annotations.json', 'r', encoding='utf-8') as f:
    all_a = json.load(f)

with open('titles.json', 'r', encoding='utf-8') as f:
    needed_titles = json.load(f)
    
annotations_dict = {}
for a in all_a:
    annotations_dict[a['title']] = a
     
SIM_TH = 0.65
texts_dict = {}
needed_titles = set(needed_titles)
encoder = SentenceTransformer('deepvk/USER-bge-m3')

with tqdm(total=496858, desc="Processing records") as pbar:
    for record in dataset:
        title = record.get("title", "")
        authors = record.get("authors", [""])
        lang = record.get("lang", "")

        if title in needed_titles:
            if title not in texts_dict:
                if lang in ['ru', 'rus']:
                    if annotations_dict[title]['author'] in authors:
                        texts_dict[title] = (record.get("sections", ""), authors)
                    else:
                        for author in authors:
                            if similarity(author, annotations_dict[title]['author']) > SIM_TH:
                                texts_dict[title] = (record.get("sections", " "), authors)
                                break

        pbar.set_postfix({
            'found': len(texts_dict),
        })
        pbar.update(1)

        if len(texts_dict) == len(needed_titles):
            break
            
combined = []
for title, value in texts_dict.items():
  combined.append({'title': title, 'author': annotations_dict[title]['author'], 'authors': value[1], 'annotation': annotations_dict[title]['annotation'],'text': value[0], 'categories': annotations_dict[title]['categories']})

Processing records: 100%|██████████| 496858/496858 [4:05:20<00:00, 33.75it/s, found=634]


In [1]:
import sys
sys.path.append('methods')

from methods import Summarisation

In [2]:
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

In [3]:
bench = Summarisation(URL=url, KEY=key, model_name='RefalMachine/RuadaptQwen3-32B-Instruct-v2') 

In [8]:
bench.change_model(key, url, 'RefalMachine/RuadaptQwen2.5-7B-Lite-Beta')

In [9]:
print(bench.blueprint.think_pass)

 no_think


In [9]:
print(bench.model_name)

DeepSeek V3


In [10]:
lst = list(await bench.client.client.models.list())[0][1]
for l in lst:
    print(l)

Model(id='RefalMachine/RuadaptQwen2.5-7B-Lite-Beta', created=None, object=None, owned_by=None, max_model_len=30000, status='spawned')
Model(id='RefalMachine/RuadaptQwen2.5-32B-Pro-Beta', created=None, object=None, owned_by=None, max_model_len=28000, status='spawned')
Model(id='RefalMachine/RuadaptQwen3-32B-Instruct-v2', created=None, object=None, owned_by=None, max_model_len=24000, status='spawned')
Model(id='llama3-70b', created=None, object=None, owned_by=None, max_model_len=14000, status='spawned')
Model(id='DeepSeek V3', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='Qwen/Qwen2.5-VL-72B-Instruct', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='Qwen3-235B-A22B', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='RefalMachine/RuadaptQwen2.5-7B-Garant-v2', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded')
Model(id='deepseek-r1-32b', cr

In [10]:
for i, j in enumerate(bench.collection[:100]):
    print(i, len('\n'.join(j['text'])))

0 60664
1 60300
2 635480
3 870892
4 17631
5 12383
6 24864
7 24373
8 10304
9 18493
10 25151
11 47887
12 122565
13 44215
14 28251
15 17376
16 16603
17 16872
18 7772
19 17818
20 11862
21 11645
22 11566
23 18891
24 3825
25 7398
26 9984
27 8546
28 9307
29 335612
30 9789
31 5568
32 3927
33 7307
34 25253
35 4284
36 6688
37 60183
38 25868
39 117459
40 483063
41 9877
42 9501
43 29465
44 411489
45 7646
46 83778
47 1537853
48 18316
49 37593
50 43823
51 8217
52 192441
53 932304
54 11062
55 21920
56 24449
57 102817
58 135059
59 82998
60 524475
61 705631
62 63766
63 49239
64 66981
65 121975
66 698463
67 52643
68 133309
69 14779
70 18277
71 37638
72 157543
73 17064
74 26297
75 3840
76 56724
77 253915
78 81049
79 53921
80 24017
81 55174
82 30742
83 51644
84 35035
85 59239
86 7965
87 18731
88 286982
89 365070
90 592985
91 345999
92 291264
93 351626
94 363080
95 313399
96 334280
97 328262
98 169556
99 24202


In [43]:
from utils import chunk_text

for i, item in enumerate(bench.collection[:100]):
    text = "\n".join(item['text'])
    chunks = chunk_text(text)
    print(i)
    print(len(text))
    print('chunks: ', len(chunks))

0
60664
chunks:  8
1
60300
chunks:  9
2
635480
chunks:  86
3
870892
chunks:  109
4
17631
chunks:  3
5
12383
chunks:  2
6
24864
chunks:  4
7
24373
chunks:  4
8
10304
chunks:  2
9
18493
chunks:  3
10
25151
chunks:  4
11
47887
chunks:  7
12
122565
chunks:  17
13
44215
chunks:  6
14
28251
chunks:  5
15
17376
chunks:  3
16
16603
chunks:  3
17
16872
chunks:  3
18
7772
chunks:  2
19
17818
chunks:  3
20
11862
chunks:  2
21
11645
chunks:  2
22
11566
chunks:  2
23
18891
chunks:  3
24
3825
chunks:  1
25
7398
chunks:  2
26
9984
chunks:  2
27
8546
chunks:  2
28
9307
chunks:  2
29
335612
chunks:  48
30
9789
chunks:  2
31
5568
chunks:  1
32
3927
chunks:  1
33
7307
chunks:  2
34
25253
chunks:  4
35
4284
chunks:  1
36
6688
chunks:  2
37
60183
chunks:  8
38
25868
chunks:  4
39
117459
chunks:  16
40
483063
chunks:  56
41
9877
chunks:  2
42
9501
chunks:  2
43
29465
chunks:  5
44
411489
chunks:  61
45
7646
chunks:  2
46
83778
chunks:  12
47
1537853
chunks:  197
48
18316
chunks:  3
49
37593
chunks:  6
50
43

In [4]:
from utils import chunk_text

text = "\n".join(bench.collection[78]['text'])
chunks = chunk_text(text)
print(len(text))
print('chunks: ', len(chunks))

81049
chunks:  11


In [5]:
model_names = ['RefalMachine/RuadaptQwen3-32B-Instruct-v2', 'DeepSeek V3', 'tpro', 'yagpt5lite']

In [7]:
import time

for name in model_names:
    print(name)
    bench.change_model(key, url, name)
    a1 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.blueprint.run(chunks, 500, 'default')
        end = time.perf_counter()
        c = end - start
        a1 += c
    res = a1 / 3
    print(f'Blueprint: {res:.2f}')
    a2 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.blueprint.run(chunks, 500, 'cluster')
        end = time.perf_counter()
        c = end - start
        a2 += c
    res = a2 / 3
    print(f'Blueprint cluster: {res:.2f}')
    a3 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.hierarchical.run(chunks, 500, False)
        end = time.perf_counter()
        c = end - start
        a3 += c
    res = a3 / 3
    print(f'Hierarchical: {res:.2f}')
    a4 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.hierarchical.run(chunks, 500, True)
        end = time.perf_counter()
        c = end - start
        a4 += c
    res = a4 / 3
    print(f'Hierarchical filtered: {res:.2f}')
    print()

RefalMachine/RuadaptQwen3-32B-Instruct-v2


CancelledError: 

In [4]:
bench.change_model(key, url, 'RefalMachine/RuadaptQwen3-32B-Instruct-v2')

In [ ]:
bench.change_model(key, url, 'DeepSeek V3')

In [ ]:
bench.change_model(key, url, 'tpro')

In [ ]:
bench.change_model(key, url, 'yagpt5lite')

In [39]:
bench.change_model(key, url, 'RefalMachine/RuadaptQwen2.5-7B-Lite-Beta')

In [22]:
import time

In [47]:
a1 = 0
for _ in range(3):
    start = time.perf_counter()
    s = await bench.blueprint.run(chunks, 500, 'default')
    end = time.perf_counter()
    c = end - start
    a1 += c
    print(f'Blueprint: {c:.2f}')
print(a1 / 3)

Blueprint: 96.89
Blueprint: 107.47
Blueprint: 106.65
103.66953063333374


In [48]:
a2 = 0
for _ in range(3):
    start = time.perf_counter()
    s = await bench.blueprint.run(chunks, 500, 'cluster')
    end = time.perf_counter()
    c = end - start
    a2 += c
    print(f'Blueprint cluster: {c:.2f}')
print(a2 / 3)

Blueprint cluster: 74.62
Blueprint cluster: 88.15
Blueprint cluster: 74.21
78.99428309999955


In [49]:
a3 = 0
for _ in range(3):
    start = time.perf_counter()
    s = await bench.hierarchical.run(chunks, 500, False)
    end = time.perf_counter()
    c = end - start
    a3 += c
    print(f'Hierarchical: {c:.2f}')
print(a3 / 3)

Hierarchical: 85.62
Hierarchical: 77.69
Hierarchical: 90.60
84.63721289999982


In [51]:
a4 = 0
for _ in range(3):
    start = time.perf_counter()
    s = await bench.hierarchical.run(chunks, 500, True)
    end = time.perf_counter()
    c = end - start
    a4 += c
    print(f'Hierarchical filtered: {c:.2f}')
print(a4 / 3)

Hierarchical filtered: 24.76
Hierarchical filtered: 26.49
Hierarchical filtered: 25.86
25.702083133333264


In [9]:
print(s1)

### Краткое содержание:

**Эмоциональные реакции Мейзи:**
- Мейзи испытывает смесь восторга от красоты и магии Волвердена, особенно восхищаясь елизаветинской архитектурой, но испытывает глубокий страх и беспокойство перед башней и старушкой Бесси, что отражает её недоверие к мистическим аспектам и предчувствие надвигающейся опасности.

**Характеристики Волвердена:**
- Замок, совмещающий готические и елизаветинские элементы, символизирует защиту и магическую силу, защищённый трёхслойной магией и древними символами, что подчеркивает его мистическую значимость и древнее наследие.

**Значение старухи Бесси:**
- Старуха Бесси, хранительница древних знаний, играет ключевую роль в местной мифологии, её присутствие в башне Волвердена усиливает мистическую атмосферу, вызывая одновременно страх и глубокое уважение среди обитателей, будучи связующей нитью с прошлым Братства.

**Развитие событий и переживаний Мейзи:**
- Путь Мейзи от первоначального восторга к глубокому посвящению в Братство включ

In [6]:
print(s)

В начале XX века, в живописных окрестностях кентского Вильде, где каждый камень хранит древние тайны, разворачивается история, связанная с таинственным замком Волверден, недавно обретшим второе дыхание благодаря стараниям его нового хозяина – полковника Уэст. Этот величественный особняк, где архитектура эпохи Елизаветы изящно переплетается с изысканными элементами, становится не просто жилищем, а живым зеркалом столетий, окутанным легендами и мистикой.

В центр повествования попадает Мейзи Льюэллин, дочь антиквара, чья глубокая связь с прошлым и страстью к антиквариату словно магнит притягивает её к Волвердену. Приезд Мейзи в замок, организованный заботливой миссис Уэст, хозяйкой, преданной традициям и хранительницей старинного очага, знаменует начало её погружения в атмосферу, насыщенную предчувствием и мистикой.

С первых шагов Мейзи увлекает реставрация замка, особенно таинственная башня, символизирующая не столько физическую реконструкцию, сколько попытку сохранить хрупкие нити про

In [7]:
await bench.evaluater.evaluate_annotation(bench.collection[0]['annotation'], s)

['Кто является главной героиней истории и куда она приглашена перед Рождеством?', 'Какова история башни Волверден и почему ее пришлось реконструировать?', 'Кто такая старуха Бесси и что она рассказывает Мэйзи о башне Волверден?', 'Кто такие две загадочные девушки, которых Мэйзи встречает на ужине, и что они представляют собой?', 'Что происходит с Мэйзи, когда она идет на прогулку с подругами и попадает в склеп?', 'Какова цель ритуала, в котором Мэйзи становится участницей, и что ожидается от нее?', 'Кто спасает Мэйзи от участи стать третьей жертвой и что происходит с башней Волверден в конце истории?']


['Двадцатилетняя дочь антиквара Мэйзи Льюэллин приглашена погостить в замок елизаветинской эпохи Волверден-холл в окрестностях Лондона перед Рождеством.', 'Башня Волверден была старинной башней, которая, согласно легенде, была трижды скреплена и укреплена душами девушек, чтобы противостоять натиску людей и дьявола. Однако со временем башня пришла в плачевное состояние и ее пришлось рек

((np.float32(0.64318556), np.float32(0.54767674), np.float32(0.59160113)),
 0.4393,
 0.2857142857142857,
 0.2365714285714286)

In [10]:
await bench.evaluater.evaluate_annotation(bench.collection[0]['annotation'], s1)

['Кто является главной героиней истории и куда она приглашена перед Рождеством?', 'Какова история старинной башни Волверден и почему её пришлось реконструировать?', 'Кто такая старуха Бесси и что она рассказывает Мэйзи о башне Волверден?', 'Что происходит с Мэйзи во время ужина и живых картин в замке Волверден-холл?', 'Кто такие Иоланта и Гедда, и как они появляются в жизни Мэйзи?', 'Что происходит с Мэйзи, когда она идет на прогулку с Иолантой и Геддой, и куда они ее ведут?', 'Что такое Братство Мёртвых Зодчих и Жертв пределов Волвердена, и какова их роль в истории?', 'Почему Мэйзи соглашается стать третьей жертвой, и что происходит с ней на верхней площадке башни?', 'Кто спасает Мэйзи от жертвы, и что происходит с ней после этого?', 'Что происходит с башней Волверден в конце истории, и как Мэйзи реагирует на это событие?']


['Двадцатилетняя дочь антиквара Мэйзи Льюэллин приглашена погостить в замок елизаветинской эпохи Волверден-холл в окрестностях Лондона перед Рождеством.', 'Стари

((np.float32(0.6147925), np.float32(0.51682734), np.float32(0.56156945)),
 0.3215,
 0.1,
 0.06949999999999999)

In [5]:
from utils import chunk_text

for item in bench.collection[:2]:
    text = '\n'.join(item['text'])
    chunks = chunk_text(text)
    s = await bench.blueprint.run(chunks, initial_word_limit=500)
    print('Blueprint')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))
    s = await bench.cluster_blueprint.run(chunks, initial_word_limit=500)
    print('Cluster blueprint')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))
    s = await bench.hierarchical.run(chunks, initial_word_limit=500)
    print('Hierarchical')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))
    s = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=True)
    print('Hierarchical filtered')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))

Blueprint
((np.float32(0.65899706), np.float32(0.5885342), np.float32(0.62177575)), 0.4988, 0.5714285714285714, 0.454)
Cluster blueprint
((np.float32(0.62056816), np.float32(0.53078747), np.float32(0.57217735)), 0.3997, 0.1111111111111111, 0.081)
Hierarchical
((np.float32(0.6443127), np.float32(0.55432045), np.float32(0.59593827)), 0.4336, 0.5714285714285714, 0.43842857142857145)
Hierarchical filtered
((np.float32(0.6438272), np.float32(0.55141735), np.float32(0.59404993)), 0.4317, 0.5714285714285714, 0.441)
Blueprint
((np.float32(0.57740545), np.float32(0.5320905), np.float32(0.5538226)), 0.2276, 0.2727272727272727, 0.1969090909090909)
Cluster blueprint
((np.float32(0.54771286), np.float32(0.49291065), np.float32(0.51886874)), 0.3262, 0.18181818181818182, 0.12854545454545455)
Hierarchical
((np.float32(0.56958234), np.float32(0.4970916), np.float32(0.5308737)), 0.3647, 0.1, 0.09290000000000001)
Hierarchical filtered
((np.float32(0.5903005), np.float32(0.51581717), np.float32(0.55055106

In [13]:
from utils import chunk_text
from tqdm import tqdm
import time
import asyncio

final_result = []
total_count = 0

with tqdm(total = 100, desc='Summarizing') as pbar:
    for item in bench.collection:
        if total_count == 100:
            print('DONE')
            break
            
        for _ in range(3):
            try:
                
                tmp = {}
                text = '\n'.join(item['text'])
                if len(text) > 800000:
                    break
                chunks = chunk_text(text)
                s = await bench.blueprint.run(chunks, initial_word_limit=500, mode='default')
                tmp['blueprint'] = s
                pbar.set_postfix({
                'blueprint': len(s),
                })
                
                s = await bench.blueprint.run(chunks, initial_word_limit=500, mode='cluster')
                tmp['blueprint_cluster'] = s
                pbar.set_postfix({
                'blueprint_cluster': len(s),
                })
                
                s = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=False)
                tmp['hierarchical'] = s
                pbar.set_postfix({
                'hierarchical': len(s),
                })
                
                s = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=True)
                tmp['hierarchical_filtered'] = s
                pbar.set_postfix({
                'hierarchical_filtered': len(s),
                })
                
                final_result.append(
                    {'title': item['title'], 
                     'author': item['author'], 
                     'authors': item['authors'], 
                     'annotations': tmp, 
                     'gold_annotation': 
                     item['annotation']}
                )
                total_count += 1
                pbar.update(1)
                break
                
            except (KeyboardInterrupt, asyncio.CancelledError):
                raise
                
            except:
                time.sleep(10)
                continue

Summarizing:   0%|          | 0/100 [17:16<?, ?it/s]


CancelledError: 

In [13]:
for i, item in enumerate(bench.collection[:100]):
    print(len('\n'.join(item['text'])))

60664
60300
635480
870892
17631
12383
24864
24373
10304
18493
25151
47887
122565
44215
28251
17376
16603
16872
7772
17818
11862
11645
11566
18891
3825
7398
9984
8546
9307
335612
9789
5568
3927
7307
25253
4284
6688
60183
25868
117459
483063
9877
9501
29465
411489
7646
83778
1537853
18316
37593
43823
8217
192441
932304
11062
21920
24449
102817
135059
82998
524475
705631
63766
49239
66981
121975
698463
52643
133309
14779
18277
37638
157543
17064
26297
3840
56724
253915
81049
53921
24017
55174
30742
51644
35035
59239
7965
18731
286982
365070
592985
345999
291264
351626
363080
313399
334280
328262
169556
24202


In [12]:
f2 = final_result

In [15]:
bench.collection[36]['annotation']

'Идёт суд над босым и худым мужичком Денисом Григорьевым. Его обвиняют в отвинчивании гайки, которой рельсы крепятся к шпалам. Мужичок этого не отрицает, но своей вины не видит. Следователь выясняет, что Денис, как и другие климовские мужики, откручивает гайки для того, чтобы делать из них грузила. Подсудимый искренне не понимает, что такое отвинчивание может привести к аварии поезда и к гибели людей. Следователь отправляет злоумышленника в тюрьму, но тот по прежнему не понимает, что он такого сделал.'

In [17]:
import json

with open('llama_final.json', 'w', encoding='utf-8') as f:
    json.dump(final_result, f, indent=4, ensure_ascii=False)

In [9]:
print(len(final_result))

19


In [15]:
print(len(f2))

63
